# rSNAPed : RNA Sequence to NAscent Protein Experiment Designer. 

___

**rSNAPed** is a library to simulate single-molecule gene expression experiments to test machine learning and computational pipelines. The code generates simulated intensity translation spots using [rSNAPsim](https://github.com/MunskyGroup/rSNAPsim). Cell segmentation is performed using [Cellpose](https://github.com/MouseLand/cellpose). Spot detection and tracking is achieved using [Trackpy](http://soft-matter.github.io/trackpy/dev/index.html). If you use `rSNAPed`, please make sure you properly cite `cellpose`, `trackpy` and `rSNAPsim`.

___

Please cite as:
___
Luis Aguilera, William Raymond, Tatsuya Morisaki, Brooke Silagy, Timothy J. Stasevich, & Brian Munsky. (2022). rSNAPed. RNA Sequence to NAscent Protein Experiment Designer. (v0.1-beta.2). Zenodo. https://doi.org/10.5281/zenodo.6967555

---

```
Author: Luis Aguilera
Contact Info: luis.aguilera@colostate.edu

Copyright (c) 2022 
Luis Aguilera, Will Raymond, 
Tim Stasevich, Brian Munsky,
Colorado State University.
Licensed under BSD-3-Clause license.
```

# Ethical Considerations and Content Policy
___
You must accept our Content Policy when using this library: 

* All simulated images generated with this software are intended to be used to test Machine Learning or computational algorithms. 
* All images generated with this software should always be labeled with the specific term "simulated images".
* All datasets resulting from a simulated image should explicitly be reported with the term "simulated data".
* Under any circumstance, a simulated image or dataset generated with rSNAPed should not be used to misrepresent real data.
* For public or private use, you must clearly disclose that the generated images are simulated and give proper credit to rSNAPed.

By running this notebook you accept our Content Policy. 

# Importing rSNAPed

____

In [ ]:
""" #@title Installing libraries - This will take some minutes
%%capture

!pip uninstall opencv_python_headless -y
!pip install "opencv-python-headless<4.3"
!pip install cellpose
!pip install 'imgaug==0.2.5'
!pip install 'trackpy>=0.5.0' 
!pip install 'seaborn>=0.11.2
!pip install 'bqplot>=0.12.21'
!pip install 'tifffile==2021.11.2'
!pip install 'snapgene_reader>=0.1.18'
!pip install 'joblib>=1.0.1'
!pip install 'pandas>=1.1.5'
!pip install 'ipywidgets>=7.6.3'
!pip install 'tqdm>=4.56.2'
!pip install 'imageio>=2.9.0'
!pip install 'ipympl==0.8.5'
!pip install 'croparray>=0.1.2'
!pip install biopython
!apt install libeigen3-dev
!ln -sf /usr/include/eigen3/Eigen /usr/include/Eigen
!pip install 'rsnapsim-ssa-cpp==0.0.20'
!pip install 'rsnapsim==0.0.47'

# Importing libraries
import os; from os import listdir; from os.path import isfile, join
import re  
import matplotlib.pyplot as plt 
from skimage.io import imread        # Module from skimage to read images as numpy arrays
import numpy as np 
from tqdm.notebook import tqdm
from timeit import default_timer as timer
import scipy
import pandas as pd
import shutil
import pathlib
import sys
import seaborn as sns
import rsnapsim as rss
import scipy.stats as stats
import random
from random import randrange
from matplotlib.animation import FuncAnimation, PillowWriter
from dna_features_viewer import BiopythonTranslator
from croparray import crop_array_tools as ca 

# Defining directories
current_dir = pathlib.Path().absolute()
rsnaped_dir = current_dir.joinpath('rsnaped')
sequences_dir = rsnaped_dir.joinpath('DataBases','gene_files')
video_dir = rsnaped_dir.joinpath('DataBases','videos_for_sim_cell')
gene_file = rsnaped_dir.joinpath('DataBases','gene_files','KDM5B_withTags.txt')
masks_dir = rsnaped_dir.joinpath('DataBases','masks_for_sim_cell')

# cloning the rSNAPed repository 
!git clone --depth 1 https://github.com/MunskyGroup/rsnaped.git
sys.path.append(str(rsnaped_dir.joinpath('rsnaped')))
import rsnaped as rsp """


In [ ]:
import os; from os import listdir; from os.path import isfile, join
import re  
import matplotlib.pyplot as plt 
from skimage.io import imread        # Module from skimage to read images as numpy arrays
import numpy as np 
from tqdm.notebook import tqdm
from timeit import default_timer as timer
import scipy
import pandas as pd
import shutil
import pathlib
import sys
import seaborn as sns
import rsnapsim as rss
import scipy.stats as stats
import random
from random import randrange
from matplotlib.animation import FuncAnimation, PillowWriter
from dna_features_viewer import BiopythonTranslator
from croparray import crop_array_tools as ca 


In [ ]:
# Defining directories
current_dir = pathlib.Path().absolute()
sequences_dir = current_dir.parents[1].joinpath('DataBases','gene_files')
video_dir = current_dir.parents[1].joinpath('DataBases','videos_for_sim_cell')
trajectories_dir = current_dir.parents[1].joinpath('DataBases','rsnapsim_simulations','kdm5b_ssa.npy')
rsnaped_dir = current_dir.parents[1].joinpath('rsnaped')
gene_file = current_dir.parents[1].joinpath('DataBases','gene_files','KDM5B_withTags.txt')
masks_dir = current_dir.parents[1].joinpath('DataBases','masks_for_sim_cell')

In [ ]:
# Importing rSNAPed
sys.path.append(str(rsnaped_dir))
import rsnaped as rsp

# Gene Construct

____

In [ ]:
#@title Function to plot DNA sequence with the library [dna_features_viewer](https://edinburgh-genome-foundry.github.io/DnaFeaturesViewer/)
plt.style.use('default')
plt.rcParams['figure.dpi'] = 120
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['axes.grid'] = False

class MyCustomTranslator(BiopythonTranslator):
    """Custom translator
    """
    def compute_feature_color(self, feature):
        if feature.type == "CDS":
            return "#57B956"
        elif feature.type == "FLAG":
            return "#ff0000"
        elif feature.type == "MS2":
            return "#098BF5"
        elif feature.type == "PP7": 
            return "#EB5559"
        else:
            return "#C4B07B"

In [ ]:
""" #@title Reading the GenBank file
gene_file_pUB_SM_KDM5B_PP7 = str(sequences_dir.joinpath('kdm5b.gb')) # plasmid pUB_SM_KDM5B_PP7

count = 0 
with open(gene_file_pUB_SM_KDM5B_PP7) as file:
    for line in file:
       print(line)
       count = count + 1 """

In [ ]:
# Plasmid sequences
gene_file_KDM5B = str(sequences_dir.joinpath('kdm5b.gb')) 
graphic_record = MyCustomTranslator().translate_record(gene_file_KDM5B) 
ax, _ = graphic_record.plot(figure_width=20, strand_in_label_threshold=7)
graphic_record.plot_legend(ax=ax, loc=1, ncol=3, frameon=False)
name_figure = 'sequence.png'

# Simulating single-molecule translation experiments

____

[//]: <> ( <img src= https://github.com/MunskyGroup/rsnaped/raw/master/docs/images/presentation/presentation_rsnaped_colab/Slide4.png alt="drawing" width="1200"/>)

### Parameters for the simulation
___

In [ ]:
# These are the parameters that need to be tested. 
number_spots_per_cell = 80     
intensity_calculation_method = 'disk_donut'  # options are : 'total_intensity' and 'disk_donut' 'gaussian_fit'
frame_selection_empty_video = 'generate_from_gaussian' #'linear_interpolation' # Options are: 'constant' , 'shuffle', 'loop', 'linear_interpolation', 'generate_from_poisson', 'generate_from_gaussian'
simulated_RNA_intensities = 'constant'  # optiions are 'constant' or 'random_values'

particle_size = 7              # spot size for the simulation and tracking.
elongation_rate = 20           # reported values 0.1 to 15  
initiation_rate = 0.1          # reported values 0.03
diffusion_coefficient = 2      # reported values 0.5 px^s 

simulation_time_in_sec = 400   # 0.55
# Perturbations
use_Harringtonin=0
perturbation_time_start = 0
perturbation_time_stop = 0

# Intensity scale
intensity_scale_ch0 = 800
intensity_scale_ch1 = 800
intensity_scale_ch2 = 0

In [ ]:
#@title ####Function to simulate cell

def fun_simulated_cells(current_dir, video_dir,masks_dir=None,ke=3,ki=0.03,gene_file =None, trajectories_dir=None, number_of_simulated_cells=3,number_spots_per_cell=80,
    simulation_time_in_sec =100,step_size_in_sec=1,particle_size=5, diffusion_coefficient =1, path_to_save_output='temp',
    intensity_calculation_method='gaussian_fit',frame_selection_empty_video=frame_selection_empty_video,
    perturbation_time_start=perturbation_time_start,perturbation_time_stop=perturbation_time_stop,
    use_Harringtonin=use_Harringtonin,save_as_gif=False,simulated_RNA_intensities='constant' ,dataframe_format='long'):
    spot_size = particle_size
    spot_sigma = 1
    # Code that creates the folder to store results.
    diffusion_coefficient_string = str(diffusion_coefficient).replace('.','_')
    directory_name = 'Simulation_'+'ns_'+str(number_spots_per_cell) +'_diff_'+ diffusion_coefficient_string 
    path_to_save_output = 'temp'
    save_to_path =  current_dir.joinpath(path_to_save_output , directory_name )
    if not os.path.exists(str(save_to_path)):
        os.makedirs(str(save_to_path))
    else:
        shutil.rmtree(str(save_to_path))
        os.makedirs(str(save_to_path))
    counter = 0
    ## Main loop that creates each cell and dataframe
    for cell_number in range (0, number_of_simulated_cells):
        output_directory_name = str(video_dir)
        list_files_names = sorted([f for f in listdir(output_directory_name) if isfile(join(output_directory_name, f)) and ('.tif') in f], key=str.lower)  # reading all tif files in the folder
        list_files_names.sort(key=lambda f: int(re.sub('\D', '', f)))  # sorting the index in numerical order
        path_files = [ str(video_dir.joinpath(f).resolve()) for f in list_files_names ] # creating the complete path for each file
        selected_video = 0 #randrange(len(path_files))
        video_path = path_files[selected_video]        
        video = imread(video_path) 
        # Loading the mask image
        if not (masks_dir is None):
            mask_image=imread(masks_dir.joinpath('mask_cell_shape_'+str(selected_video)+'.tif'))       
        else:
            mask_image=None
        # Reducing in a half the intensity in the original video
        empty_videos = video
        counter +=1
        if counter>=len(path_files):
            counter =0
        if not (trajectories_dir is None ):
            # Loading trajectories from file
            ssa_trajectories = np.load(str(trajectories_dir))
            random_index_ch0 = np.random.randint(low=0, high=ssa_trajectories.shape[0]-1, size=(number_spots_per_cell,))
            random_index_ch1 = np.random.randint(low=0, high=ssa_trajectories.shape[0]-1, size=(number_spots_per_cell,))
            random_index_ch2 = np.random.randint(low=0, high=ssa_trajectories.shape[0]-1, size=(number_spots_per_cell,))
            
            simulated_trajectories_ch0 = ssa_trajectories[random_index_ch0,0:simulation_time_in_sec:step_size_in_sec] / 10 # converting to ump
            simulated_trajectories_ch1 = ssa_trajectories[random_index_ch1,0:simulation_time_in_sec:step_size_in_sec] / 10 # converting to ump
            simulated_trajectories_ch2 =  ssa_trajectories[random_index_ch2,0:simulation_time_in_sec:step_size_in_sec] / 10 # converting to ump
        else:
            # Simulations for intensity
            ssa1,ssa1_ump,_ = rsp.SSA_rsnapsim(gene_file,ke,ki,frames=simulation_time_in_sec,frame_rate=1,n_traj=number_spots_per_cell,use_Harringtonin=use_Harringtonin,use_FRAP=0, 
                                               perturbation_time_start=perturbation_time_start,perturbation_time_stop=perturbation_time_stop).simulate() 
            simulated_trajectories_ch1 = ssa1_ump
            ssa2,ssa2_ump,_ =  rsp.SSA_rsnapsim(gene_file,ke,ki,frames=simulation_time_in_sec,frame_rate=1,n_traj=number_spots_per_cell,use_Harringtonin=use_Harringtonin,use_FRAP=0, 
                                                perturbation_time_start=perturbation_time_start,perturbation_time_stop=perturbation_time_stop).simulate() 
            simulated_trajectories_ch2 = ssa2_ump
            simulated_trajectories_ch0 = rsp.SimulateRNA(shape_output_array=(number_spots_per_cell, simulation_time_in_sec), rna_intensity_method=simulated_RNA_intensities,
                                                         min_int=0,max_int=5,mean_int=5 ).simulate()
        #simulated_trajectories_ch0 = None
        # Running the cell simulation
        saved_file_name = str(save_to_path.joinpath('sim_cell_'+str(cell_number)))
        _ , _, _ = rsp.SimulatedCell( base_video=video, mask_image=mask_image, number_spots = number_spots_per_cell, number_frames=simulation_time_in_sec, step_size=step_size_in_sec, 
                                     diffusion_coefficient =diffusion_coefficient, simulated_trajectories_ch0=simulated_trajectories_ch0, size_spot_ch0=spot_size, spot_sigma_ch0=spot_sigma, 
                                     simulated_trajectories_ch1=simulated_trajectories_ch1, size_spot_ch1=spot_size, spot_sigma_ch1=spot_sigma, simulated_trajectories_ch2=simulated_trajectories_ch2, 
                                     size_spot_ch2=spot_size, spot_sigma_ch2=spot_sigma, ignore_ch0=0,ignore_ch1=0, ignore_ch2=0,save_as_tif_uint8=0,save_as_tif =1,save_as_gif=save_as_gif, save_dataframe=1,
                                     saved_file_name=saved_file_name,create_temp_folder = False, intensity_calculation_method=intensity_calculation_method,perform_video_augmentation=0,
                                     frame_selection_empty_video=frame_selection_empty_video ,intensity_scale_ch0 = intensity_scale_ch0,intensity_scale_ch1 = intensity_scale_ch1,
                                     intensity_scale_ch2 = intensity_scale_ch2,dataframe_format=dataframe_format).make_simulation()      
        print ('The results are saved in folder: ', saved_file_name)
    return save_to_path, simulated_trajectories_ch0, simulated_trajectories_ch1, simulated_trajectories_ch2, empty_videos

In [ ]:
# running the simulation
start = timer()
output_directory_name, _, _, _, _ = fun_simulated_cells(current_dir,video_dir,masks_dir=masks_dir,ke=elongation_rate, 
                                                                    ki=initiation_rate,gene_file= gene_file, 
                                                                    number_of_simulated_cells=1,number_spots_per_cell=number_spots_per_cell,
                                                                    simulation_time_in_sec =simulation_time_in_sec,step_size_in_sec=1,particle_size=particle_size, 
                                                                    diffusion_coefficient=diffusion_coefficient,intensity_calculation_method=intensity_calculation_method,
                                                                    frame_selection_empty_video=frame_selection_empty_video,perturbation_time_start=perturbation_time_start,
                                                                    perturbation_time_stop=perturbation_time_stop,use_Harringtonin=use_Harringtonin,
                                                                    simulated_RNA_intensities=simulated_RNA_intensities)
end = timer()
print('Time to generate simulated data:',round(end - start), ' sec')

#### Display results as images and dataframe
____

In [ ]:
# Loading the simulated video
simulated_video = imread(str(output_directory_name)+'/sim_cell_0.tif')

### Plotting the simulated image




In [ ]:
#@title Plotting the simulated cells
selected_time_point = 1
max_percentile=99.95
print('Dimensions in the simulated video: ', simulated_video.shape)
fig, ax = plt.subplots(3,2, figsize=(10, 7))
int_red = simulated_video[selected_time_point,:,:,0]
int_green = simulated_video[selected_time_point,:,:,1]
int_blue = simulated_video[selected_time_point,:,:,2]

# Red
ax[0,0].imshow(int_red,cmap=plt.cm.Greys,vmax=np.percentile(int_red,max_percentile))
ax[0,0].set_title('Channel 0 = RNA');ax[0,0].set_xticks([]); ax[0,0].set_yticks([])
ax[0,1].hist(int_red.flatten(), bins=80,color='red')
ax[0,1].set_xlabel('Intensity'); ax[0,1].set_ylabel('Count')

# Green
ax[1,0].imshow(int_green,cmap=plt.cm.Greys,vmax=np.percentile(int_green,max_percentile))
ax[1,0].set_title('Channel 1 = Protein 1'); ax[1,0].set_xticks([]); ax[1,0].set_yticks([])
ax[1,1].hist(int_green.flatten(), bins=80,color='green')
ax[1,1].set_xlabel('Intensity'); ax[1,1].set_ylabel('Count')

# Blue
ax[2,0].imshow(int_blue,cmap=plt.cm.Greys,vmax=np.percentile(int_blue,max_percentile))
ax[2,0].set_title('Channel 2 = Protein 2'); ax[2,0].set_xticks([]); ax[2,0].set_yticks([])
ax[2,1].hist(int_blue.flatten(), bins=80,color='blue')
ax[2,1].set_xlabel('Intensity'); ax[2,1].set_ylabel('Count')
plt.show()

In [ ]:
""" # @title Visualizing the simulated video
max_frames = simulated_video.shape[0]
reduced_video = simulated_video[0:max_frames,:,:,:]
fig,axes = plt.subplots(1,3,dpi=120,figsize=(10,3))
i=0
# Define inital frames
Red = reduced_video[i,:,:,0]
im1 = axes[0].imshow(Red,cmap='Greys_r')
Green = reduced_video[i,:,:,1]
im2 = axes[1].imshow(Green,cmap='Greys_r')
Blue = reduced_video[i,:,:,2]
im3 =  axes[2].imshow(Blue,cmap='Greys_r')
axes[0].axis('off')
axes[1].axis('off')
axes[2].axis('off')
axes[0].set_title('Channel 0')
axes[1].set_title('Channel 1')
axes[2].set_title('Channel 2')

def movieFrame(i):
  Red = reduced_video[i,:,:,0]
  Green = reduced_video[i,:,:,1]
  Blue = reduced_video[i,:,:,2]
  images = [Red,Green,Blue]
  image_handles = [im1,im2,im3]
  for k,image_n in enumerate(images):
    image_handles[k].set_array(images[k])
  return image_handles
  
plt.close()
anim = FuncAnimation(fig, movieFrame, frames=max_frames, interval=60, blit=True)

#anim.save("simulated_cell_1.gif", dpi=300, writer=PillowWriter(fps=5))

from IPython.display import HTML
HTML(anim.to_html5_video()) """

In [ ]:
df = pd.read_csv(str(output_directory_name)+'/sim_cell_0_df.csv')
df

In [ ]:
df.columns

# Signal to Noise Ratio (SNR). 

___

The SNR is a metric to calculate the 'strength' of the signal over the noise in the image.

$SNR = \frac{I_{in}-I_{out}}{\sigma_{out}}$,

where, is the intensity in the translation site $I_{in}$, is the intensity in an area outside the translation site $I_{out}$,and $\sigma_{OUT}$ is the standard deviation outside the translation site.

In [ ]:
SNR_red_channel = df.SNR_red.values
SNR_green_channel = df.SNR_green.values

In [ ]:
#@title ####Plotting SNR

# Plotting
fig, ax = plt.subplots(1,2, figsize=(10, 3))
ax[0].hist(SNR_red_channel, bins = 40, color = 'orangered')
ax[0].set_ylabel('Count')
ax[0].set_title('$SNR_{Red}$')
ax[0].set_xlabel('SNR')

ax[1].hist(SNR_green_channel, bins = 40, color = 'limegreen')
ax[1].set_ylabel('Count')
ax[1].set_title('$SNR_{Green}$')
ax[1].set_xlabel('SNR')

plt.show()

In [ ]:
raise

#Mean Square Displacement (MSD). 

___

MSD is a metric to calculate the average movement of particles with respect to time. [MSD formal definition](https://en.wikipedia.org/wiki/Mean_squared_displacement).

</br>
${\rm MSD}=\frac{1}{N}\sum_{i=1}^N |\mathbf{x^{(i)}}(t) - \mathbf{x^{(i)}}(0)|^2 $
</br>


</br>
where $N$ is the number of particles, $\mathbf{x^{(i)}}(0)$ is the reference position of the $i^{th}$ particle, and $\mathbf{x^{(i)}}(t)$ is the position of the $i^{th}$ particle at time $t$.

</br>

In a random motion process, there is a linear relationship between the MSD and time $t$. [Link with the derivation of this formula](https://en.wikipedia.org/wiki/Mean_squared_displacement).
</br>


$MSD= 2nDt$,
</br>

where $n$ is the number of dimensions,  $D$ is the  diffusion coefficient, 
</br>

$D= \frac{k_B \cdot T}{6 \pi \eta r}$ (Einstein-Stokes equation)

*   Proportional to: $k_B$ Boltzman constant, $T$ absolute temperature
*   Inversely Proportional: $\eta$ is the viscocity, $r$ particle radius.

For more deails please check this [notebook](https://colab.research.google.com/drive/18jXJi3D7OvRftR8Y6KmZK3Do0syPP2ij?usp=sharing)

In [ ]:
#@title #### Function to calculate msd
def compute_msd(trajectory):
  '''
  This function is intended to calculate the mean square displacement of a given trajectory.
  msd(τ)  = <[r(t+τ) - r(t)]^2>
  Inputs:
    trajectory: list of temporal evolution of a centers of mass .  [Y_val_particle_i_tp_0, X_val_particle_i_tp_0]   , ... , [Y_val_particle_i_tp_n, X_val_particle_i_tp_n] ]

  Returns
    msd: float, mean square displacement
    rmsd: float, root mean square displacement
  '''
  total_length_trajectory=len(trajectory)
  msd=[]
  for i in range(total_length_trajectory-1):
      tau=i+1
      # Distance that a particle moves for each time point (tau) divided by time
      # msd(τ)                 = <[r(t+τ)  -    r(t)]^2>
      msd.append(np.sum((trajectory[0:-tau]-trajectory[tau::])**2)/float(total_length_trajectory-tau)) # Reverse Indexing 
  # Converting list to np.array
  msd=np.array(msd)   # array with shape Nspots vs time_points
  rmsd = np.sqrt(msd)
  return msd, rmsd 

In [ ]:
# Calculating the MSD
t=np.arange(1,simulation_time_in_sec,1)
msd_trajectories = np.zeros((number_spots_per_cell,simulation_time_in_sec-1))
rmsd_trajectories = np.zeros((number_spots_per_cell,simulation_time_in_sec-1))
for i in range(0,number_spots_per_cell):
  # extracting the particle positions from the dataframe.
  particle_trajectory = df[['y','x']] [df['particle'] == i].values
  msd_trajectories[i,:], rmsd_trajectories[i,:] = compute_msd(particle_trajectory)

In [ ]:
# MSD Statistics (mu, sigma) for all trajectories.
msd_trajectories_all_mu = np.mean(msd_trajectories,axis=0)
msd_trajectories_all_sem = np.std(msd_trajectories,axis=0) /np.sqrt(number_spots_per_cell)

In [ ]:
# calculating the Diffusion coefficient
# k_dif = msd / (2 * n * t) 
k_diff_calulated = msd_trajectories_all_mu/(2*t)

In [ ]:
#@title ####Plotting the MSD vs Time
downsampling = 20

fig, ax = plt.subplots(1,2, figsize=(10, 3))

for i in range(0,number_spots_per_cell):
  ax[0].plot(msd_trajectories[i,:],'-',color=[0.8,0.8,0.8])
ax[0].errorbar(t[::downsampling], msd_trajectories_all_mu[::downsampling],  yerr=msd_trajectories_all_sem[::downsampling],ecolor='dodgerblue',linestyle='')
ax[0].plot(t[::downsampling], msd_trajectories_all_mu[::downsampling], marker='o', markersize=5, linestyle='-',color='dodgerblue',label ='mean' )
ax[0].legend()
ax[0].set_title('Mean square displacement')
ax[0].set_ylabel('MSD ($px^2$)')
ax[0].set_xlabel('time (s)')
ax[0].set_ylim((0,3000))

ax[1].plot(t[::downsampling], k_diff_calulated[::downsampling], marker='o', markersize=5, linestyle='-',color='dodgerblue',label ='mean' )
ax[1].legend()
ax[1].set_title('Diffusion coefficient')
ax[1].set_ylabel('D ($px^{2}$/sec)')
ax[1].set_xlabel('time (s)')


plt.show()

[//]: <> ( <img src= https://github.com/MunskyGroup/uqbio2022/raw/master/files/files_image_processing/module_1_3/images/Slide6.png alt="drawing" width="600"/> )

In [ ]:
#@title Plotting trajectories of intensity in the video

intensity_values_in_image = np.zeros((number_spots_per_cell,simulation_time_in_sec)) # pre-allocating memory for intensity
for i in range(number_spots_per_cell):
  intensity_values_in_image[i,:] = df[df['particle'] ==i].green_int_mean.values
  
print('simulated trajectories shape: ', intensity_values_in_image.shape)

fig, ax = plt.subplots(1,2, figsize=(10, 2.5), gridspec_kw = {'width_ratios': [4, 1]})
ax[0].plot(intensity_values_in_image.T,'grey',alpha = .1)
ax[0].plot(intensity_values_in_image[25,:].T,'-', linewidth = 1, color = 'limegreen',label = 'representative')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Intensity (au)')
ax[0].set_ylim((intensity_values_in_image.min(),intensity_values_in_image.max()))
ax[0].set_title('Intensities from simulated video', color ='k')
ax[0].legend()
ax[1].hist(intensity_values_in_image[:,selected_time_point] , bins = 40, color = 'limegreen', orientation = 'horizontal')
ax[1].set_ylim((intensity_values_in_image.min(),intensity_values_in_image.max()))
ax[1].set_xlabel('Count')
plt.subplots_adjust(wspace=0.15, hspace=0)
plt.show()

In [ ]:
#@title Plotting trajectories of intensity from the SSA

# Generating a large sample from the SSA-TASEP model
number_trajectories = 400
selected_time_point = 0
ssa, ssa_ump, _ = rsp.SSA_rsnapsim(gene_file,ke=elongation_rate,ki=initiation_rate,frames=simulation_time_in_sec,frame_rate=1,n_traj=number_trajectories,use_Harringtonin=use_Harringtonin,use_FRAP=0, perturbation_time_start=perturbation_time_start,perturbation_time_stop=perturbation_time_stop).simulate() # rss.ssa_solver(n_traj = number_spots_per_cell, start_time=starting_time,tf=starting_time+n_frames, tstep=starting_time+n_frames,k_elong_mean=3, k_initiation=.03)  # tstep = total number of steps including the burnin time 
print('ssa shape: ', ssa_ump.shape)

fig, ax = plt.subplots(1,2, figsize=(10, 2.5), gridspec_kw = {'width_ratios': [4, 1]})
ax[0].plot(ssa_ump.T,'grey',alpha = .1)
ax[0].plot(ssa_ump[25,:].T,'-', linewidth = 2, color = 'k',label = 'representative')
ax[0].set_xlabel('Time (s)')
ax[0].set_ylabel('Intensity (ump)')
ax[0].set_ylim((ssa_ump.min(),ssa_ump.max()))
ax[0].set_title('Intensities from  SSA - TASEP model', color ='k')
ax[0].legend()
ax[1].hist(ssa_ump[:,selected_time_point] , bins = 40, color = 'k', orientation = 'horizontal')
ax[1].set_ylim((ssa_ump.min(),ssa_ump.max()))
ax[1].set_xlabel('Count')
plt.subplots_adjust(wspace=0.15, hspace=0)
plt.show()

In [ ]:
#@title ####Code to calculate the auto-correlation function.
def get_autocorrelation(data, g0='G0', norm='individual'):
    n_traj = data.shape[0]
    acf_vec = np.zeros(data.shape)

    def get_acc_fft(signal):
        N = len(signal)
        fvi = np.fft.fft(signal, n=2*N)
        acf = fvi*np.conjugate(fvi)
        acf = np.fft.ifft(acf)
        acf = np.real(acf[:N])/float(N)
        return acf

    global_mean = np.mean(data)
    global_var = np.var(data)
    for i in range(n_traj):
        if norm == 'individual':
            if np.mean(data[i] == 0):
                signal = (data[i] - 1e-6) / 1e-6
            else:
                signal = (data[i] - np.mean(data[i])) / np.var(data[i])
        else:
            signal = (data[i] - global_mean) / global_var
        if g0 == 'G1':
            g1 = get_acc_fft(signal)[1]
            acf_vec[i] = get_acc_fft(signal)/g1
        if g0 == 'G0':
            g = get_acc_fft(signal)[0]
            acf_vec[i] = get_acc_fft(signal)/g
    return acf_vec

# Autocorrelation function ($G(\tau)$).

____
Correlation between a signal with respect of itself at diferent time intervals ($\tau$)

$G(\tau) = \frac{\operatorname{E} \left[(I_{t+\tau} - \mu_{I})(I_{t} - \mu_{I})\right]}{\sigma_{I}^2}$


In [ ]:
#@title ####  Plotting the auto-correlation function


# Calculating the auto-correlation function for SSA
acf_from_ssa = get_autocorrelation(ssa_ump, g0='G0', norm='none')
mean_acf_ssa = np.mean(acf_from_ssa, axis=0)
err_acf_ssa = np.std(acf_from_ssa, axis=0)  

acf_from_data = get_autocorrelation(intensity_values_in_image, g0='G0', norm='none')
mean_acf_data = np.mean(acf_from_data, axis=0)
err_acf_data = np.std(mean_acf_data, axis=0) 

#max_x_val = np.min((400, len(mean_acf_ssa)))

# Plotting the ACF and its mean.
fig, ax = plt.subplots(1,2, figsize=(10, 3))
# Plotting ACF for SSA
tau = np.arange(0, len(mean_acf_ssa), 1)
ax[0].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax[0].plot(tau, mean_acf_ssa, '-', linewidth=3, color='k', label='mean ACF')
ax[0].fill_between(tau, mean_acf_ssa - err_acf_ssa, mean_acf_ssa + err_acf_ssa, color='grey', alpha=0.1)
ax[0].set_ylim((-0.2, 1))
ax[0].set_xlabel('tau')
ax[0].set_ylabel(r'G/G(0)')
ax[0].set_title('SSA', color ='k')
# Plotting the ACF for Image
ax[1].axhline(y=0, color='k', linestyle='-', linewidth=0.5)
ax[1].plot(tau, mean_acf_data, '-', linewidth=3, color='limegreen', label='mean ACF')
ax[1].fill_between(tau, mean_acf_data - err_acf_data, mean_acf_data + err_acf_data, color='grey', alpha=0.1)
ax[1].set_ylim((-0.2, 1))
ax[1].set_xlabel('tau')
ax[1].set_ylabel(r'G/G(0)')

ax[1].set_title('Image', color ='k')

try:
  tau_ssa = tau[np.where(mean_acf_ssa<0)[0][0]]
  print('The dwell (decorrelation) time from the SSA is ', str(tau_ssa) , 'seconds')
  ax[0].plot([tau_ssa,tau_ssa], [0,0.25], '--', linewidth=1, color='orangered', label= r'$\tau_{ssa}$')

  tau_image = tau[np.where(mean_acf_data<0)[0][0]]
  print('The dwell (decorrelation) time from the image is ', str(tau_image) , 'seconds')
  ax[1].plot([tau_image,tau_image], [0,0.25], '--', linewidth=1, color='orangered', label= r'$\tau_{Image}$')
except:
  print('It was not possible to estimate the dwell (decorrelation) times.')
  tau_ssa =1
  tau_image =1

ax[0].legend()
ax[1].legend()

plt.show()


Make sure to discuss about the [shot noise](https://en.wikipedia.org/wiki/Shot_noise). Ask Brian, Tim or Tatsuya to elaborate. 

Coulon, Antoine, et al. "Kinetic competition during the transcription cycle results in stochastic RNA processing." Elife 3 (2014): e03939.[Link to publication](https://elifesciences.org/articles/03939)


## Calculating Elongation ($k_{e}$) rates from the $G(\tau)$

_____

$\tilde{k_{e}} = \frac{L}{\tau_{dw}} $,

where $L$ is the gene length in codons, and $\tau_{dw}$ is the dwell time.

In [ ]:
protein_strs = rss.seqmanip.open_seq_file(str(gene_file))[0] # getting the protein length from the gene file.
protein_length = len(protein_strs['1'][0])
print ('The calculated elongation rate in the SSA is:   ',   str(np.round(protein_length/tau_ssa,1)), ' (aa/sec) \n') 
print ('The calculated elongation rate in the image is: ', str(np.round(protein_length/tau_image,1)), ' (aa/sec) \n'  )

#### Removing outliers

In [ ]:
def remove_extrema(vector ,max_percentile = 99):
    '''This function is intended to remove extrema data given by the min and max percentiles specified by the user'''
    max_val = np.percentile(vector, max_percentile)
    new_vector = vector [vector< max_val] 
    return new_vector

### Normalizing data

$ X_{norm} = \frac{X -min(X)}{max(X) - min(X)} $

In [ ]:
# Normalizing data
selected_time_point = 0
data1_raw = ssa_ump[:,selected_time_point].flatten() 
data1 = (data1_raw- np.min(data1_raw)) / (np.max(data1_raw)-np.min(data1_raw))
data_sorted_1 = np.sort(data1)
p_1 =np.linspace(0, 1, len(data1), endpoint=False)

data2_raw = remove_extrema(intensity_values_in_image[:,selected_time_point].flatten(), max_percentile = 98)
data2 = (data2_raw - np.min(data2_raw)) / (np.max(data2_raw)-np.min(data2_raw))
data_sorted_2 = np.sort(data2)
p_2 =np.linspace(0, 1, len(data2), endpoint=False)

## Comparing empirical distributions between the SSA and the intensity in the image.


The empirial distribution function (EDF)

$F(X) = \frac{1}{n} ∑_{i=1}^{n} I_{(X_{i} \le x)}$

$I_{A}$ is the indicator function of the event $A$.

The empirical cumulative distribution function (eCDF) of random variable X

$F_{X}(x) = P(X \le x) $.


Definitions from [Ramachandran, 2020].


In [ ]:
#@title Plotting empirical distribution functions

# Plotting
fig, ax = plt.subplots(1,3, figsize=(15, 4))
ax[0].hist(data_sorted_1, bins = 40, color = 'k')
ax[0].set_ylabel('Count')
ax[0].set_title('EDF ($\hat{I_{SSA}}$)')
ax[1].hist(data_sorted_2, bins = 40, color = 'limegreen')
ax[1].set_ylabel('Count')
ax[1].set_title('EDF ($\hat{I_{image}}$)')
ax[2].plot(data_sorted_1, p_1,linewidth=3,label ='SSA',color ='k')
ax[2].plot(data_sorted_2, p_2,linewidth=3,label ='Image',color ='limegreen')
ax[2].set_title('eCDF')
ax[2].set_ylabel('Frequency')
ax[2].set_xlabel('Normalized intensity')
ax[2].legend()
plt.show()

#### Metric to compare distributions

The Kolmogorov distance $D_{KS}$ is the maximum vertical distance between two eCDF $F_1$ and $F_2$, with $n$ and $m$ samples, respectively.

$D_{KS}=\max_x |F_{1,n}(x)-F_{2,m}(x)|$

In [ ]:
# Calculating Kolmogorov distance between the two ECDF.
ks_distance = scipy.stats.kstest(data_sorted_1,data_sorted_2).statistic
print('The KS-distance between SSA and image is:' , round(ks_distance,2))

# Calculating ribosome initiation rates ($k_{i}$).
_____
$\tilde{k_{i}} ≈ \frac{\mu_{I}}{\tau_{dw}}$,

where $\mu_{I}$ is the mean intensity in units of mature protein (UMP), and $\tau_{dw}$ is the dwell time calculated from the autocorrelation function.

From Eq 21 in [Aguilera 2019 paper] (https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007425)

In [ ]:
mean_int_ssa = np.mean(data1_raw)
calculated_initiation_rate_ssa = np.round(mean_int_ssa /tau_ssa ,2)
print('The calculated intensity rate for the SSA is: ', calculated_initiation_rate_ssa ,' 1/sec')

To relate fluorescence units to units of mature proteins (UMP) we need to determine a scaling factor. To see more about this, please check [Morisaki et al, 2016. Real-time quantification of single RNA translation dynamics in living cells](https://www.science.org/doi/10.1126/science.aaf0899)


In [ ]:
# Determining this scaling factor requires performing an experiment with a single tag and then determining the absolute intensity obtained from a single tagged spot
number_probes_in_construct = 10
intensity_in_single_tagged_spot = 10 # assumed number.
scaling_factor = (number_probes_in_construct * intensity_in_single_tagged_spot)

# Converting fluorescence units to units of mature proteins (UMP). 
mean_int_image = np.mean(data2_raw)
mean_int_image_ump =mean_int_image/scaling_factor
calculated_initiation_rate_image = np.round(mean_int_image_ump /tau_image,2)
print('The calculated intensity rate for the image is: ', calculated_initiation_rate_image ,' 1/sec ')

In [ ]:
raise

# Quality Control
_____

### Visualizing spots using CropArray


To use croparray we need to use the simulated video and a dataframe with the particle spositions.

In [ ]:
# Shape in the simulated video (f, y, x, ch). 
simulated_video.shape

In [ ]:
#@title #### Converting the video and dataframe into Croparray fromat.
# For croparray the dimensions MUST be (fov, f , z, y, x, ch)
img_croparray = np.expand_dims(simulated_video,axis=(0,2)) 
img_croparray.shape 

# Renaming some colums and adding values to the new columns
spots = df.copy() # Creating a copy of the dataframe
# renaming some columns
spots['id']=spots.index 
spots.rename(columns={'x': 'xc','y': 'yc', 'frame': 'f'}, inplace=True, errors='raise')
# adding column with fov.
spots['fov']= 0
spots.rename(columns={'particle':'id'})
spots = spots[['fov','id','f','yc','xc']]  

# Creating the croparray
my_ca = ca.create_crop_array(img_croparray,spots,xy_pad=3)
#print(my_ca.coords , '\n')
#print(my_ca.dims)

# Changing the format to best_z. 
best_z = ca.best_z_proj(my_ca, ref_ch=0, disk_r=3, roll_n=1)
#print(best_z.coords , '\n')
#print(best_z.dims)

# View the action of montage
my_ca_montage= ca.montage(best_z, row = 'n', col = 't')
#print(my_ca_montage.dims)
#print(my_ca_montage.coords)

In [ ]:
#@title Plotting croparray for channel 0
montage_ch0  = my_ca_montage.sel(ch=0).values
plt.figure(figsize=(30,5))
plt.imshow(montage_ch0[0],cmap='Greys_r')
plt.xlabel('Time (s)', size=10)
plt.ylabel('Particles', size=10)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
#@title Plotting croparray for channel 1

montage_ch1  = my_ca_montage.sel(ch=1).values
plt.figure(figsize=(30,5))
plt.imshow(montage_ch1[0],cmap='Greys_r')
plt.xlabel('Time (s)', size=10)
plt.ylabel('Particle', size=10)
plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
#@title Plotting croparray for channel 2

montage_ch2  = my_ca_montage.sel(ch=2).values
plt.figure(figsize=(30,5))
plt.imshow(montage_ch2[0],cmap='Greys_r')
plt.xlabel('Time (s)', size=10)
plt.ylabel('Particle', size=10)
plt.xticks([])
plt.yticks([])
plt.show()

## Testing for Bleed-through (crosstalk) of fluorescence emission spectra.

In [ ]:
#@title ###Extracting data from pandas dataframe

def extracting_data(dataframe,selected_field='green_int_mean', remove_negative_values=False,remove_extreme_values=False,selected_time=0):
    extracted_data = dataframe.loc[(dataframe['frame']==selected_time)][selected_field].values
    # Negative values may appear if the intensity in the background is higher than the intensity in the spot. This option allows to remove negative values.
    if remove_negative_values == True:
        extracted_data =  extracted_data[extracted_data>=0] 
    # To plot the histogram, we could remove extreme values. Extreme values are defined as those above the 98 percentile. 
    if remove_extreme_values==True:
        extracted_data= remove_extrema(extracted_data)  
    # Option to normalize data to the maximum value.
    normalized_extracted_data = (extracted_data-np.min(extracted_data))/ (np.max(extracted_data)-np.min(extracted_data))  
    return extracted_data,normalized_extracted_data

In [ ]:
#@title Function for scatter plot

def plot_scatter_spots_cell_size(x,y,plot_title,selected_color = '#1C00FE',xlabel='',ylabel='',max_percentile=95):
    
    df_join_distribution = pd.DataFrame({'X':x,'Y':y})

    # Removing outliers as values that are above 98% percentile
    df_join_distribution.sort_values(by=['X'], ascending=True,inplace=True)
    print('Scatter plot removing values that are avobe the ',str(max_percentile),' percentile')
    len_df = len (df_join_distribution)
    df_join_distribution_wo_outliers = df_join_distribution.head(int(len_df*(max_percentile/100)))
    
    r, p = stats.pearsonr(df_join_distribution_wo_outliers.X.values, df_join_distribution_wo_outliers.Y.values)
    
    plt.figure(figsize=(4, 4))
    sns.set(font_scale = 1.3)
    b = sns.jointplot(data=df_join_distribution_wo_outliers, y='Y', x='X', color= selected_color , marginal_kws=dict(bins=40, rug=True))
    b.plot_joint(sns.rugplot, height=0, color=[0.7,0.7,0.7], clip_on=True)
    b.plot_joint(sns.kdeplot, color=[0.5,0.5,0.5], levels=5)
    b.plot_joint(sns.regplot,scatter_kws={'color': 'orangered',"s":10, 'marker':'o'}, line_kws={'color': selected_color,'lw': 2} )
    blank_plot, = b.ax_joint.plot([], [], linestyle="", alpha=0)
    b.ax_joint.legend([blank_plot],['r={:.2f}'.format( np.round(r,2))],loc='upper left',)
    b.ax_joint.set_xlim(np.percentile(x,1), np.percentile(x,max_percentile))
    b.ax_joint.set_ylim(np.percentile(y,1), np.percentile(y,max_percentile))
    b.fig.suptitle(plot_title)
    b.ax_joint.set_xlabel(xlabel)
    b.ax_joint.set_ylabel(ylabel)
    b.ax_joint.collections[0].set_alpha(0)
    b.fig.tight_layout()
    b.fig.subplots_adjust(top=0.92) 
    plt.show()
    return

### Comparing intenisty between the Red and Green channels

In [ ]:
# Scatter plots intensity comparing all channels
int_red,_ =  extracting_data(dataframe=df, selected_field='red_int_mean')
int_green,_ =  extracting_data(dataframe=df,selected_field='green_int_mean')
plot_scatter_spots_cell_size(int_red,int_green,plot_title='Red vs Green',selected_color = '#1C00FE',xlabel='red intensity',ylabel='green intensity',max_percentile=97)

# References
_____

[1] Ramachandran, K.M. and Tsokos, C.P., 2020. Mathematical statistics with applications in R. Academic Press.

Katz, Zachary B., et al. "Mapping translation'hot-spots' in live cells by tracking single molecules of mRNA and ribosomes." Elife 5 (2016): e10415.


Morisaki, T., Lyon, K., DeLuca, K.F., DeLuca, J.G., English, B.P., Zhang, Z., Lavis, L.D., Grimm, J.B., Viswanathan, S., Looger, L.L. and Lionnet, T., 2016. Real-time quantification of single RNA translation dynamics in living cells. Science, 352(6292), pp.1425-1429.
